***DOWNLOADING DEPENDENCIES***

In [ ]:
!pip install opencv-python opencv-python-headless matplotlib tqdm kagglehub
!pip install ultralytics
!pip install huggingface_hub ultralytics
!pip install scrfd --quiet
!pip install scrfd
!pip install -q onnxruntime opencv-python-headless numpy tqdm
!pip install -q insightface onnxruntime opencv-python-headless tqdm
!pip install -q onnxruntime
!pip install insightface
!pip install onnxruntime opencv-python
!git clone https://github.com/yakhyo/facial-analysis.git
!bash facial-analysis/download.sh
!wget -O scrfd_500m.onnx "https://sourceforge.net/projects/insightface.mirror/files/v0.7/scrfd_person_2.5g.onnx/download"
!wget -O scrfd_10g_bnkps.onnx \
  https://huggingface.co/lithiumice/insightface/resolve/main/models/antelopev2/scrfd_10g_bnkps.onnx

***DOWNLOADING MODELS***
1. YOLOv8n-FACE
2. SCRFD

In [ ]:
#Downloading pre-trained YOLOv8n-Face Model

from huggingface_hub import hf_hub_download
from ultralytics import YOLO

model_path = hf_hub_download(
    repo_id="arnabdhar/YOLOv8-Face-Detection",
    filename="model.pt"
)
print("Model downloaded to:", model_path)

model = YOLO(model_path)
print("Model architecture loaded successfully!")

***DOWNLOADING DATASETS***
1. FDDB
2. WIDERFACE
3. CELEBA

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("cormacwc/fddb-dataset")

print("Path to dataset files:", path)

In [ ]:
import kagglehub
import shutil
import os

# Download dataset
src_path = kagglehub.dataset_download("mksaad/wider-face-a-face-detection-benchmark")
print("Downloaded to:", src_path)

# Writable destination path
dest_path = "/kaggle/working/wider-face-dataset"
os.makedirs(dest_path, exist_ok=True)

# Copy contents to working dir
shutil.copytree(src_path, dest_path, dirs_exist_ok=True)

print("Dataset copied to:", dest_path)

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/celeba-dataset")

print("Path to dataset files:", path)

***--FDDB DATASET--***

**Haar Cascades on FDDB**



In [ ]:
# FDDB Face Detection with Haar Cascades + Accuracy Evaluation

# STEP1: Set up paths for already uploaded FDDB dataset
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

# Set the base paths for images and folds
base_folder = "/kaggle/input/fddb-dataset/FDDB copy/originalPics"
folds_path = "/kaggle/input/fddb-dataset/FDDB copy/FDDB-folds"

# STEP 2: Helper functions
frontal_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')

def load_image(path):
    img = cv2.imread(path)
    if img is None:
        raise ValueError(f"Image not found: {path}")
    return img

def detect_faces_haar_combined(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    frontal_faces = frontal_cascade.detectMultiScale(gray, 1.1, 5)
    profile_faces = profile_cascade.detectMultiScale(gray, 1.1, 5)
    flipped = cv2.flip(gray, 1)
    flipped_profiles = profile_cascade.detectMultiScale(flipped, 1.1, 5)
    fixed_profiles = [(gray.shape[1] - x - w, y, w, h) for (x, y, w, h) in flipped_profiles]
    return list(frontal_faces) + list(profile_faces) + fixed_profiles

def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    return interArea / float(boxAArea + boxBArea - interArea) if boxAArea + boxBArea - interArea > 0 else 0

def ellipse_to_bbox(major, minor, angle, cx, cy, _):
    x1 = int(cx - major)
    y1 = int(cy - minor)
    w = int(2 * major)
    h = int(2 * minor)
    return [x1, y1, w, h]

# STEP 3: Load GT annotations
gt_annotations = {}
for i in range(1, 11):
    with open(f"{folds_path}/FDDB-fold-{i:02d}-ellipseList.txt") as f:
        while True:
            name = f.readline().strip()
            if not name:
                break
            count = int(f.readline())
            boxes = []
            for _ in range(count):
                vals = list(map(float, f.readline().strip().split()))
                boxes.append(ellipse_to_bbox(*vals))
            filename = name + ".jpg"
            gt_annotations[filename] = boxes

# STEP 4: Detection + Evaluation
TP = FP = FN = 0
IOU_THRESH = 0.5

for root, dirs, files in os.walk(base_folder):
    for file in tqdm(files):
        if file.endswith('.jpg'):
            relative_path = os.path.relpath(os.path.join(root, file), base_folder)
            key = relative_path.replace("\\", "/")
            img_path = os.path.join(root, file)
            img = load_image(img_path)
            preds = detect_faces_haar_combined(img)
            gt = gt_annotations.get(key, [])
            matched = set()
            for p in preds:
                matched_flag = False
                for i, g in enumerate(gt):
                    if i in matched:
                        continue
                    if iou(p, g) >= IOU_THRESH:
                        TP += 1
                        matched.add(i)
                        matched_flag = True
                        break
                if not matched_flag:
                    FP += 1
            FN += len(gt) - len(matched)

# STEP 5: Report Metrics
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print(f"TP: {TP}, FP: {FP}, FN: {FN}")
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")


**Haar Cascades (with NMS + Tuned Parameters) on FDDB**

In [ ]:
# FDDB Face Detection with Haar Cascades + Accuracy Evaluation (with NMS + Tuned Parameters)

# STEP 1: Set up paths for already uploaded FDDB dataset
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

# Set the base paths for images and folds
base_folder = "/kaggle/input/fddb-dataset/FDDB copy/originalPics"
folds_path = "/kaggle/input/fddb-dataset/FDDB copy/FDDB-folds"

# STEP 2: Helper functions
frontal_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')

def load_image(path):
    img = cv2.imread(path)
    if img is None:
        raise ValueError(f"Image not found: {path}")
    return img

def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []
    boxes = np.array(boxes)
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,0] + boxes[:,2]
    y2 = boxes[:,1] + boxes[:,3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = idxs[-1]
        pick.append(last)
        xx1 = np.maximum(x1[last], x1[idxs[:-1]])
        yy1 = np.maximum(y1[last], y1[idxs[:-1]])
        xx2 = np.minimum(x2[last], x2[idxs[:-1]])
        yy2 = np.minimum(y2[last], y2[idxs[:-1]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / area[idxs[:-1]]
        idxs = np.delete(idxs, np.concatenate(([len(idxs) - 1], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

def detect_faces_haar_combined(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    frontal_faces = frontal_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=6, minSize=(30, 30))
    profile_faces = profile_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=6, minSize=(30, 30))
    flipped = cv2.flip(gray, 1)
    flipped_profiles = profile_cascade.detectMultiScale(flipped, scaleFactor=1.2, minNeighbors=6, minSize=(30, 30))
    fixed_profiles = [(gray.shape[1] - x - w, y, w, h) for (x, y, w, h) in flipped_profiles]
    all_faces = list(frontal_faces) + list(profile_faces) + fixed_profiles
    return non_max_suppression_fast(all_faces, 0.3)

def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    return interArea / float(boxAArea + boxBArea - interArea) if boxAArea + boxBArea - interArea > 0 else 0

def ellipse_to_bbox(major, minor, angle, cx, cy, unused):
    x1 = int(cx - major)
    y1 = int(cy - minor)
    w = int(2 * major)
    h = int(2 * minor)
    return [x1, y1, w, h]

# STEP 3: Load GT annotations
gt_annotations = {}
for i in range(1, 11):
    with open(f"{folds_path}/FDDB-fold-{i:02d}-ellipseList.txt") as f:
        while True:
            name = f.readline().strip()
            if not name:
                break
            count = int(f.readline())
            boxes = []
            for _ in range(count):
                vals = list(map(float, f.readline().strip().split()))
                boxes.append(ellipse_to_bbox(*vals))
            filename = name + ".jpg"
            gt_annotations[filename] = boxes

# STEP 4: Detection + Evaluation
TP = FP = FN = 0
IOU_THRESH = 0.5

for root, dirs, files in os.walk(base_folder):
    for file in tqdm(files):
        if file.endswith('.jpg'):
            relative_path = os.path.relpath(os.path.join(root, file), base_folder)
            key = relative_path.replace("\\", "/")
            img_path = os.path.join(root, file)
            img = load_image(img_path)
            preds = detect_faces_haar_combined(img)
            gt = gt_annotations.get(key, [])
            matched = set()
            for p in preds:
                matched_flag = False
                for i, g in enumerate(gt):
                    if i in matched:
                        continue
                    if iou(p, g) >= IOU_THRESH:
                        TP += 1
                        matched.add(i)
                        matched_flag = True
                        break
                if not matched_flag:
                    FP += 1
            FN += len(gt) - len(matched)

# STEP 5: Report Metrics
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print(f"TP: {TP}, FP: {FP}, FN: {FN}")
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")


**YOLOV8-FACE on FDDB**

In [ ]:
# FDDB DATASET RUN WITH YOLOV8-FACE MODEL:

# STEP 1: Import Libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tqdm import tqdm

# STEP 2: Set FDDB paths
base_folder = "/kaggle/input/fddb-dataset/FDDB copy/originalPics"
folds_path = "/kaggle/input/fddb-dataset/FDDB copy/FDDB-folds"

# Load model from HuggingFace download path
model_path = "/root/.cache/huggingface/hub/models--arnabdhar--YOLOv8-Face-Detection/snapshots/52fa54977207fa4f021de949b515fb19dcab4488/model.pt"
model = YOLO(model_path)

# STEP 3: Load Ground Truth Ellipses → Convert to BBoxes
def ellipse_to_bbox(major, minor, angle, cx, cy, score=None):
    x1 = int(cx - major)
    y1 = int(cy - minor)
    w = int(2 * major)
    h = int(2 * minor)
    return [x1, y1, w, h]

gt_annotations = {}
for i in range(1, 11):
    with open(f"{folds_path}/FDDB-fold-{i:02d}-ellipseList.txt") as f:
        while True:
            name = f.readline().strip()
            if not name:
                break
            count = int(f.readline())
            boxes = []
            for _ in range(count):
                vals = list(map(float, f.readline().strip().split()))
                boxes.append(ellipse_to_bbox(*vals))
            filename = name + ".jpg"
            gt_annotations[filename] = boxes

# STEP 4: Define IoU
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    return interArea / float(boxAArea + boxBArea - interArea) if boxAArea + boxBArea - interArea > 0 else 0

# STEP 5: Evaluate on FDDB
TP = FP = FN = 0
IOU_THRESH = 0.5

for root, dirs, files in os.walk(base_folder):
    for file in tqdm(files):
        if file.endswith('.jpg'):
            rel_path = os.path.relpath(os.path.join(root, file), base_folder)
            key = rel_path.replace("\\", "/")
            img_path = os.path.join(root, file)
            img = cv2.imread(img_path)

            # Run YOLOv8 inference
            results = model.predict(img, verbose=False)
            preds = []
            for r in results:
                for box in r.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    w, h = x2 - x1, y2 - y1
                    preds.append([int(x1), int(y1), int(w), int(h)])

            gt = gt_annotations.get(key, [])
            matched = set()

            for p in preds:
                matched_flag = False
                for i, g in enumerate(gt):
                    if i in matched:
                        continue
                    if iou(p, g) >= IOU_THRESH:
                        TP += 1
                        matched.add(i)
                        matched_flag = True
                        break
                if not matched_flag:
                    FP += 1
            FN += len(gt) - len(matched)

# STEP 6: Compute Metrics
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print(f"YOLOv8-Face Evaluation Results on FDDB:")
print(f"TP: {TP}, FP: {FP}, FN: {FN}")
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")


**YOLOv8-Face (Improved with confidence tuning and filtering) on FDDB**

In [ ]:
# FDDB DATASET RUN WITH YOLOv8-FACE MODEL (Improved with confidence tuning and filtering):

# STEP 1: Import Libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tqdm import tqdm

# STEP 2: Set FDDB paths
base_folder = "/kaggle/input/fddb-dataset/FDDB copy/originalPics"
folds_path = "/kaggle/input/fddb-dataset/FDDB copy/FDDB-folds"

# Load model from HuggingFace download path
model_path = "/root/.cache/huggingface/hub/models--arnabdhar--YOLOv8-Face-Detection/snapshots/52fa54977207fa4f021de949b515fb19dcab4488/model.pt"
model = YOLO(model_path)

# STEP 3: Load Ground Truth Ellipses → Convert to BBoxes
def ellipse_to_bbox(major, minor, angle, cx, cy, score=None):
    x1 = int(cx - major)
    y1 = int(cy - minor)
    w = int(2 * major)
    h = int(2 * minor)
    return [x1, y1, w, h]

gt_annotations = {}
for i in range(1, 11):
    with open(f"{folds_path}/FDDB-fold-{i:02d}-ellipseList.txt") as f:
        while True:
            name = f.readline().strip()
            if not name:
                break
            count = int(f.readline())
            boxes = []
            for _ in range(count):
                vals = list(map(float, f.readline().strip().split()))
                boxes.append(ellipse_to_bbox(*vals))
            filename = name + ".jpg"
            gt_annotations[filename] = boxes

# STEP 4: Define IoU
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    return interArea / float(boxAArea + boxBArea - interArea) if boxAArea + boxBArea - interArea > 0 else 0

# STEP 5: Evaluate on FDDB
TP = FP = FN = 0
IOU_THRESH = 0.5
CONF_THRESH = 0.4
MIN_BOX_SIZE = 20

for root, dirs, files in os.walk(base_folder):
    for file in tqdm(files):
        if file.endswith('.jpg'):
            rel_path = os.path.relpath(os.path.join(root, file), base_folder)
            key = rel_path.replace("\\", "/")
            img_path = os.path.join(root, file)
            img = cv2.imread(img_path)

            # Run YOLOv8 inference with adjusted confidence threshold
            results = model.predict(img, conf=CONF_THRESH, verbose=False)
            preds = []
            for r in results:
                for box in r.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    w, h = x2 - x1, y2 - y1
                    if w > MIN_BOX_SIZE and h > MIN_BOX_SIZE:
                        preds.append([int(x1), int(y1), int(w), int(h)])

            gt = gt_annotations.get(key, [])
            matched = set()

            for p in preds:
                matched_flag = False
                for i, g in enumerate(gt):
                    if i in matched:
                        continue
                    if iou(p, g) >= IOU_THRESH:
                        TP += 1
                        matched.add(i)
                        matched_flag = True
                        break
                if not matched_flag:
                    FP += 1
            FN += len(gt) - len(matched)

# STEP 6: Compute Metrics
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print(f"YOLOv8-Face Evaluation Results on FDDB:")
print(f"TP: {TP}, FP: {FP}, FN: {FN}")
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")


**YOLOv8-Face (Confidence Filtering + Optional NMS) on FDDB**

In [ ]:
# Improved FDDB Evaluation with YOLOv8-Face: Confidence Filtering + Optional NMS

# STEP 1: Import Libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tqdm import tqdm

# STEP 2: Set FDDB paths
base_folder = "/kaggle/input/fddb-dataset/FDDB copy/originalPics"
folds_path = "/kaggle/input/fddb-dataset/FDDB copy/FDDB-folds"

# Load YOLOv8-Face model from HuggingFace
model_path = "/root/.cache/huggingface/hub/models--arnabdhar--YOLOv8-Face-Detection/snapshots/52fa54977207fa4f021de949b515fb19dcab4488/model.pt"
model = YOLO(model_path)

# STEP 3: Ground Truth Ellipse → BBox
def ellipse_to_bbox(major, minor, angle, cx, cy, score=None):
    x1 = int(cx - major)
    y1 = int(cy - minor)
    w = int(2 * major)
    h = int(2 * minor)
    return [x1, y1, w, h]

gt_annotations = {}
for i in range(1, 11):
    with open(f"{folds_path}/FDDB-fold-{i:02d}-ellipseList.txt") as f:
        while True:
            name = f.readline().strip()
            if not name:
                break
            count = int(f.readline())
            boxes = []
            for _ in range(count):
                vals = list(map(float, f.readline().strip().split()))
                boxes.append(ellipse_to_bbox(*vals))
            filename = name + ".jpg"
            gt_annotations[filename] = boxes

# STEP 4: IoU calculation
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    return interArea / float(boxAArea + boxBArea - interArea) if boxAArea + boxBArea - interArea > 0 else 0

# STEP 5: Evaluate with confidence threshold and optional size filtering
TP = FP = FN = 0
IOU_THRESH = 0.5
CONF_THRESH = 0.3
MIN_SIZE = 20  # Filter out small detections

for root, dirs, files in os.walk(base_folder):
    for file in tqdm(files):
        if file.endswith('.jpg'):
            rel_path = os.path.relpath(os.path.join(root, file), base_folder)
            key = rel_path.replace("\\", "/")
            img_path = os.path.join(root, file)
            img = cv2.imread(img_path)

            results = model.predict(img, conf=CONF_THRESH, verbose=False)
            preds = []
            for r in results:
                for box in r.boxes:
                    conf = box.conf[0].item()
                    if conf < CONF_THRESH:
                        continue
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    w, h = x2 - x1, y2 - y1
                    if w < MIN_SIZE or h < MIN_SIZE:
                        continue
                    preds.append([int(x1), int(y1), int(w), int(h)])

            gt = gt_annotations.get(key, [])
            matched = set()

            for p in preds:
                matched_flag = False
                for i, g in enumerate(gt):
                    if i in matched:
                        continue
                    if iou(p, g) >= IOU_THRESH:
                        TP += 1
                        matched.add(i)
                        matched_flag = True
                        break
                if not matched_flag:
                    FP += 1
            FN += len(gt) - len(matched)

# STEP 6: Metrics
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print(f"YOLOv8-Face Evaluation Results on FDDB:")
print(f"TP: {TP}, FP: {FP}, FN: {FN}")
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")

**SCRFD on FDDB**

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image
from insightface.app import FaceAnalysis

# STEP 1: Model Setup
model_dir = os.path.expanduser("~/.insightface/models/antelopev2")
model_file = os.path.join(model_dir, "scrfd_10g_bnkps.onnx")

if not os.path.exists(model_file):
    os.makedirs(model_dir, exist_ok=True)
    !wget -O "{model_file}" https://huggingface.co/lithiumice/insightface/resolve/main/models/antelopev2/scrfd_10g_bnkps.onnx

app = FaceAnalysis(name="antelopev2", providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

# STEP 2: Dataset Paths
IMG_DIR = "/kaggle/input/fddb-dataset/FDDB copy/originalPics"
FOLD_DIR = "/kaggle/input/fddb-dataset/FDDB copy/FDDB-folds"

# STEP 3: Ground Truth Loading
def ellipse_to_bbox(a, b, ang, cx, cy, *_):
    return [int(cx - a), int(cy - b), int(2 * a), int(2 * b)]

gt = {}
for i in range(1, 11):
    path = os.path.join(FOLD_DIR, f"FDDB-fold-{i:02d}-ellipseList.txt")
    with open(path) as f:
        while True:
            name = f.readline().strip()
            if not name:
                break
            n = int(f.readline().strip())
            boxes = []
            for _ in range(n):
                parts = list(map(float, f.readline().split()))
                boxes.append(ellipse_to_bbox(*parts))
            img_name = os.path.basename(name) + ".jpg"
            gt[img_name] = boxes

# STEP 4: IoU Function
def iou(a, b):
    xA = max(a[0], b[0])
    yA = max(a[1], b[1])
    xB = min(a[0] + a[2], b[0] + b[2])
    yB = min(a[1] + a[3], b[1] + b[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    union = a[2] * a[3] + b[2] * b[3] - inter
    return inter / union if union > 0 else 0

# STEP 5: Evaluate
TP = FP = FN = 0
IOU_THRESH = 0.3

for root, dirs, files in os.walk(IMG_DIR):
    for fn in tqdm(files):
        if not fn.endswith(".jpg"):
            continue

        img_path = os.path.join(root, fn)
        img = cv2.imread(img_path)
        if img is None:
            continue

        # Run SCRFD detection
        faces = app.get(img)
        preds = [list(map(int, face.bbox)) for face in faces if face.det_score > 0.5]

        # Match predictions with ground truth
        gts = gt.get(fn, [])  # Match by filename only
        matched = set()
        for p in preds:
            hit = False
            for idx, g in enumerate(gts):
                if idx in matched:
                    continue
                if iou(p, g) >= IOU_THRESH:
                    TP += 1
                    matched.add(idx)
                    hit = True
                    break
            if not hit:
                FP += 1
        FN += len(gts) - len(matched)



# STEP 6:
precision = TP / (TP + FP + 1e-9)
recall = TP / (TP + FN + 1e-9)
f1 = 2 * precision * recall / (precision + recall + 1e-9)

print("✅ SCRFD on FDDB Results:")
print(f"TP: {TP}, FP: {FP}, FN: {FN}")
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")


***--WIDER FACE  DATASET--***

**HAAR CASCADES (FRONTAL ONLY) on WIDER FACE**

In [ ]:
# HAAR CASCADES (FRONTAL ONLY)

import cv2
import os
from tqdm import tqdm
import numpy as np

# Paths
img_base_dir = "/kaggle/working/wider-face-dataset/WIDER_val/WIDER_val/images"
save_dir = "/kaggle/working/wider_haar_preds"
os.makedirs(save_dir, exist_ok=True)

# Load Haar Cascade frontal face detector
haar_model = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Parameters
scaleFactor = 1.1
minNeighbors = 4
minSize = (30, 30)

# Collect image paths
img_paths = sorted(glob.glob(f"{img_base_dir}/*/*.jpg"))

# Run face detection
for img_path in tqdm(img_paths):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    detections = haar_model.detectMultiScale(gray, scaleFactor=scaleFactor,
                                             minNeighbors=minNeighbors, minSize=minSize)

    h, w = img.shape[:2]
    txt_path = os.path.join(save_dir, os.path.splitext(os.path.basename(img_path))[0] + ".txt")

    with open(txt_path, "w") as f:
        for (x, y, w_box, h_box) in detections:
            if w_box < 10 or h_box < 10:
                continue  # Ignore very small boxes
            x_center = (x + w_box / 2) / w
            y_center = (y + h_box / 2) / h
            w_norm = w_box / w
            h_norm = h_box / h
            f.write(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

# Evaluate Haar predictions
tp, fp, fn, precision, recall, f1 = compute_metrics("/kaggle/working/wider_haar_preds", label_save_dir)

print("🔍 Haar Cascade Evaluation on WIDER FACE")
print(f"TP: {tp}, FP: {fp}, FN: {fn}")
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")

**HAAR CASCADES (FRONTAL + SIDE) + NMS on WIDER FACE**

In [ ]:
# HAAR CASCADES (FRONTAL + SIDE) + NMS

# STEP 1: Install required packages
!pip install opencv-python-headless matplotlib tqdm

# STEP 2: Import libraries
import cv2
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict

# STEP 3: Set paths (adjust according to your mount point if needed)
IMG_DIR = "/kaggle/working/wider-face-dataset/WIDER_train/WIDER_train/images"
ANNOT_FILE = "/kaggle/working/wider-face-dataset/wider_face_split/wider_face_split/wider_face_train_bbx_gt.txt"

# STEP 4: Load Haar Cascades
frontal_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')

# STEP 5: Load Ground Truth Annotations
gt = defaultdict(list)

with open(ANNOT_FILE) as f:
    lines = iter(f.readlines())
    for img_path in lines:
        img_path = img_path.strip()
        face_count = int(next(lines).strip())
        for _ in range(face_count):
            x, y, w, h, *_ = map(int, next(lines).strip().split())
            gt[img_path].append([x, y, w, h])

# STEP 6: IOU + NMS functions
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    return interArea / float(boxAArea + boxBArea - interArea) if boxAArea + boxBArea - interArea > 0 else 0

def non_max_suppression(boxes, iou_threshold=0.3):
    if not boxes:
        return []
    boxes = np.array(boxes)
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,0] + boxes[:,2]
    y2 = boxes[:,1] + boxes[:,3]
    areas = boxes[:,2] * boxes[:,3]
    order = areas.argsort()[::-1]
    keep = []

    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        w = np.maximum(0, xx2 - xx1)
        h = np.maximum(0, yy2 - yy1)
        overlap = (w * h) / (areas[i] + areas[order[1:]] - (w * h))
        order = order[1:][overlap <= iou_threshold]
    return boxes[keep].tolist()

# STEP 7: Detection + Evaluation
TP = FP = FN = 0
IOU_THRESH = 0.5

limited_gt = dict(list(gt.items())[:1000])
for rel_path, true_boxes in tqdm(limited_gt.items(), total=len(limited_gt)):
    full_path = os.path.join(IMG_DIR, rel_path)
    img = cv2.imread(full_path)
    if img is None:
        continue
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect frontal and profile faces
    frontal = frontal_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    profile = profile_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # Flip image to catch opposite profiles
    flipped = cv2.flip(gray, 1)
    flipped_profiles = profile_cascade.detectMultiScale(flipped, 1.1, 5)
    flipped_profiles = [(gray.shape[1] - x - w, y, w, h) for (x, y, w, h) in flipped_profiles]

    all_detections = list(frontal) + list(profile) + flipped_profiles
    preds = non_max_suppression(all_detections, iou_threshold=0.3)

    matched = set()
    for p in preds:
        matched_flag = False
        for i, g in enumerate(true_boxes):
            if i in matched:
                continue
            if iou(p, g) >= IOU_THRESH:
                TP += 1
                matched.add(i)
                matched_flag = True
                break
        if not matched_flag:
            FP += 1
    FN += len(true_boxes) - len(matched)

# STEP 8: Print final results
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print("\n✅ Haar + NMS Evaluation on WIDER FACE")
print(f"TP: {TP}, FP: {FP}, FN: {FN}")
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")


**YOLOV8n-Face (without NMS) on WIDER FACE**

In [ ]:
# YOLOV8n-Face run on WIDER FACE Dataset withot NMS

# STEP 1: Install and import required libraries
!pip install -q ultralytics
from ultralytics import YOLO
import os, glob, cv2
import numpy as np
from tqdm import tqdm
from huggingface_hub import hf_hub_download

# STEP 2: Define all paths
base_dir = "/kaggle/working/wider-face-dataset"
annotation_file = os.path.join(base_dir, "wider_face_split", "wider_face_split", "wider_face_val_bbx_gt.txt")
img_base_dir = os.path.join(base_dir, "WIDER_val", "WIDER_val", "images")
label_save_dir = "/kaggle/working/wider_yolo_labels"
pred_dir = "/kaggle/working/yolo_preds"
os.makedirs(label_save_dir, exist_ok=True)
os.makedirs(pred_dir, exist_ok=True)

# STEP 3: Convert ground-truth annotations to YOLO format
with open(annotation_file, 'r') as f:
    lines = f.read().strip().split('\n')

i = 0
while i < len(lines):
    img_rel_path = lines[i].strip()
    n_faces = int(lines[i + 1].strip())
    label_lines = lines[i + 2:i + 2 + n_faces]

    img_path = os.path.join(img_base_dir, img_rel_path)
    label_path = os.path.join(label_save_dir, os.path.splitext(os.path.basename(img_path))[0] + ".txt")

    if not os.path.exists(img_path):
        i += 2 + n_faces
        continue

    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    with open(label_path, 'w') as label_file:
        for line in label_lines:
            x, y, width, height, *_ = map(float, line.strip().split())
            if width < 10 or height < 10:
                continue
            x_center = (x + width / 2) / w
            y_center = (y + height / 2) / h
            w_norm = width / w
            h_norm = height / h
            label_file.write(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
    i += 2 + n_faces

# STEP 4: Load YOLOv8-Face model
model_path = hf_hub_download(repo_id="arnabdhar/YOLOv8-Face-Detection", filename="model.pt")
model = YOLO(model_path)
print("YOLOv8-Face model loaded successfully!")

# STEP 5: Run inference on validation images
img_paths = sorted(glob.glob(f"{img_base_dir}/*/*.jpg"))

for img_path in tqdm(img_paths):  # Run on full set
    result = model.predict(img_path, conf=0.15, verbose=False)[0]
    pred_file = os.path.join(pred_dir, os.path.splitext(os.path.basename(img_path))[0] + ".txt")

    with open(pred_file, 'w') as f:
        if result.boxes is not None:
            for box in result.boxes.xywhn.cpu().numpy():
                x_center, y_center, w_box, h_box = box[:4]
                f.write(f"0 {x_center:.6f} {y_center:.6f} {w_box:.6f} {h_box:.6f}\n")

# STEP 5: Define evaluation functions
def compute_iou(box1, box2):
    def to_xyxy(box):
        xc, yc, w, h = box
        x1 = xc - w / 2
        y1 = yc - h / 2
        x2 = xc + w / 2
        y2 = yc + h / 2
        return [x1, y1, x2, y2]

    b1 = to_xyxy(box1)
    b2 = to_xyxy(box2)
    xA, yA, xB, yB = max(b1[0], b2[0]), max(b1[1], b2[1]), min(b1[2], b2[2]), min(b1[3], b2[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    box1Area = (b1[2] - b1[0]) * (b1[3] - b1[1])
    box2Area = (b2[2] - b2[0]) * (b2[3] - b2[1])
    union = box1Area + box2Area - interArea
    return interArea / union if union != 0 else 0

def compute_metrics(pred_dir, gt_dir, iou_thresh=0.5):
    TP = FP = FN = 0
    pred_files = glob.glob(os.path.join(pred_dir, "*.txt"))

    for pred_file in pred_files:
        base = os.path.basename(pred_file)
        gt_file = os.path.join(gt_dir, base)
        if not os.path.exists(gt_file):
            continue

        try:
            preds = np.loadtxt(pred_file, ndmin=2)
            gts = np.loadtxt(gt_file, ndmin=2)
        except:
            continue

        matched_gt = np.zeros(len(gts), dtype=bool)

        for pred in preds:
            best_iou, best_idx = 0, -1
            for j, gt in enumerate(gts):
                iou = compute_iou(pred[1:], gt[1:])
                if iou > best_iou:
                    best_iou, best_idx = iou, j

            if best_iou >= iou_thresh and not matched_gt[best_idx]:
                TP += 1
                matched_gt[best_idx] = True
            else:
                FP += 1

        FN += len(gts) - matched_gt.sum()

    precision = TP / (TP + FP + 1e-6)
    recall = TP / (TP + FN + 1e-6)
    f1 = 2 * precision * recall / (precision + recall + 1e-6)
    return TP, FP, FN, precision, recall, f1

# STEP 6: Run evaluation if predictions exist
if not os.path.exists(pred_dir) or len(os.listdir(pred_dir)) == 0:
    print(f"Prediction folder '{pred_dir}' is missing or empty. Please run inference first.")
else:
    tp, fp, fn, precision, recall, f1 = compute_metrics(pred_dir, label_save_dir)
    print("YOLOv8-Face Evaluation on WIDER FACE")
    print(f"TP: {tp}, FP: {fp}, FN: {fn}")
    print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")


**YOLOV8n-Face (with NMS) on WIDER FACE**

In [ ]:
# YOLOV8n-Face run on WIDER FACE Dataset with NMS

# STEP 1: Install and import required libraries
!pip install -q ultralytics
from ultralytics import YOLO
import os, glob, cv2
import numpy as np
from tqdm import tqdm
from huggingface_hub import hf_hub_download
from torchvision.ops import nms
import torch

# STEP 2: Define all paths
base_dir = "/kaggle/working/wider-face-dataset"
annotation_file = os.path.join(base_dir, "wider_face_split", "wider_face_split", "wider_face_val_bbx_gt.txt")
img_base_dir = os.path.join(base_dir, "WIDER_val", "WIDER_val", "images")
label_save_dir = "/kaggle/working/wider_yolo_labels"
pred_dir = "/kaggle/working/yolo_preds"
os.makedirs(label_save_dir, exist_ok=True)
os.makedirs(pred_dir, exist_ok=True)

# STEP 3: Convert ground-truth annotations to YOLO format
with open(annotation_file, 'r') as f:
    lines = f.read().strip().split('\n')

i = 0
while i < len(lines):
    img_rel_path = lines[i].strip()
    n_faces = int(lines[i + 1].strip())
    label_lines = lines[i + 2:i + 2 + n_faces]

    img_path = os.path.join(img_base_dir, img_rel_path)
    label_path = os.path.join(label_save_dir, os.path.splitext(os.path.basename(img_path))[0] + ".txt")

    if not os.path.exists(img_path):
        i += 2 + n_faces
        continue

    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    with open(label_path, 'w') as label_file:
        for line in label_lines:
            x, y, width, height, *_ = map(float, line.strip().split())
            if width < 10 or height < 10:
                continue
            x_center = (x + width / 2) / w
            y_center = (y + height / 2) / h
            w_norm = width / w
            h_norm = height / h
            label_file.write(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
    i += 2 + n_faces

# STEP 4: Load YOLOv8-Face model
model_path = hf_hub_download(repo_id="arnabdhar/YOLOv8-Face-Detection", filename="model.pt")
model = YOLO(model_path)
print("\u2705 YOLOv8-Face model loaded successfully!")

# STEP 5: Run inference with NMS
img_paths = sorted(glob.glob(f"{img_base_dir}/*/*.jpg"))

def apply_nms(boxes, scores, iou_threshold=0.5):
    boxes_xyxy = []
    for b in boxes:
        x_c, y_c, w, h = b
        x1 = x_c - w / 2
        y1 = y_c - h / 2
        x2 = x_c + w / 2
        y2 = y_c + h / 2
        boxes_xyxy.append([x1, y1, x2, y2])
    boxes_tensor = torch.tensor(boxes_xyxy, dtype=torch.float32)
    scores_tensor = torch.tensor(scores, dtype=torch.float32)
    keep = nms(boxes_tensor, scores_tensor, iou_threshold)
    return keep

for img_path in tqdm(img_paths):
    result = model.predict(img_path, conf=0.2, verbose=False)[0]
    pred_file = os.path.join(pred_dir, os.path.splitext(os.path.basename(img_path))[0] + ".txt")

    if result.boxes is not None and result.boxes.xywhn.numel() > 0:
        boxes = result.boxes.xywhn.cpu().numpy()
        scores = result.boxes.conf.cpu().numpy()

        # Apply NMS
        boxes = np.atleast_2d(boxes)
        scores = np.atleast_1d(scores)
        keep_indices = apply_nms(boxes, scores, iou_threshold=0.5)
        boxes = boxes[keep_indices]

        # Ensure boxes is 2D for safe iteration
        if boxes.ndim == 1:
            boxes = boxes[np.newaxis, :]
    else:
        boxes = np.empty((0, 4))

    with open(pred_file, 'w') as f:
        for box in boxes:
            x_center, y_center, w_box, h_box = box[:4]
            f.write(f"0 {x_center:.6f} {y_center:.6f} {w_box:.6f} {h_box:.6f}\n")



# STEP 6: Define evaluation functions
def compute_iou(box1, box2):
    def to_xyxy(box):
        xc, yc, w, h = box
        x1 = xc - w / 2
        y1 = yc - h / 2
        x2 = xc + w / 2
        y2 = yc + h / 2
        return [x1, y1, x2, y2]

    b1 = to_xyxy(box1)
    b2 = to_xyxy(box2)
    xA, yA, xB, yB = max(b1[0], b2[0]), max(b1[1], b2[1]), min(b1[2], b2[2]), min(b1[3], b2[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    box1Area = (b1[2] - b1[0]) * (b1[3] - b1[1])
    box2Area = (b2[2] - b2[0]) * (b2[3] - b2[1])
    union = box1Area + box2Area - interArea
    return interArea / union if union != 0 else 0

def compute_metrics(pred_dir, gt_dir, iou_thresh=0.5):
    TP = FP = FN = 0
    pred_files = glob.glob(os.path.join(pred_dir, "*.txt"))

    for pred_file in pred_files:
        base = os.path.basename(pred_file)
        gt_file = os.path.join(gt_dir, base)
        if not os.path.exists(gt_file):
            continue

        try:
            preds = np.loadtxt(pred_file, ndmin=2)
            gts = np.loadtxt(gt_file, ndmin=2)
        except:
            continue

        matched_gt = np.zeros(len(gts), dtype=bool)

        for pred in preds:
            best_iou, best_idx = 0, -1
            for j, gt in enumerate(gts):
                iou = compute_iou(pred[1:], gt[1:])
                if iou > best_iou:
                    best_iou, best_idx = iou, j

            if best_iou >= iou_thresh and not matched_gt[best_idx]:
                TP += 1
                matched_gt[best_idx] = True
            else:
                FP += 1

        FN += len(gts) - matched_gt.sum()

    precision = TP / (TP + FP + 1e-6)
    recall = TP / (TP + FN + 1e-6)
    f1 = 2 * precision * recall / (precision + recall + 1e-6)
    return TP, FP, FN, precision, recall, f1

# STEP 7: Run evaluation
if not os.path.exists(pred_dir) or len(os.listdir(pred_dir)) == 0:
    print(f"Prediction folder '{pred_dir}' is missing or empty. Please run inference first.")
else:
    tp, fp, fn, precision, recall, f1 = compute_metrics(pred_dir, label_save_dir)
    print("YOLOv8-Face Evaluation on WIDER FACE")
    print(f"TP: {tp}, FP: {fp}, FN: {fn}")
    print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")

**SCRFD ON WIDER FACE**

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from insightface.app import FaceAnalysis

# STEP 1: Initialize SCRFD
app = FaceAnalysis(name="antelopev2", providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))  # Larger input size improves detection

# STEP 2: Define Paths
IMG_DIR = "/kaggle/working/wider-face-dataset/WIDER_val/WIDER_val/images"
ANNOTATION_FILE = "/kaggle/working/wider-face-dataset/wider_face_split/wider_face_split/wider_face_val_bbx_gt.txt"

# STEP 3: Parse Annotations
gt = {}
with open(ANNOTATION_FILE, "r") as f:
    while True:
        name = f.readline().strip()
        if not name:
            break
        n = int(f.readline())
        boxes = []
        for _ in range(n):
            vals = list(map(int, f.readline().strip().split()))
            x, y, w, h = vals[:4]
            if w > 15 and h > 15:  # Skip very small faces
                boxes.append([x, y, w, h])
        gt[name] = boxes

# STEP 4: IoU Function
def iou(a, b):
    xA = max(a[0], b[0])
    yA = max(a[1], b[1])
    xB = min(a[0]+a[2], b[0]+b[2])
    yB = min(a[1]+a[3], b[1]+b[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    union = a[2]*a[3] + b[2]*b[3] - inter
    return inter / union if union > 0 else 0

# STEP 5: Evaluate on Images
TP = FP = FN = 0
IOU_THRESH = 0.1
image_count = 0
MAX_IMAGES = 3226

for rel_path, gts in tqdm(gt.items(), desc="Evaluating images"):
    if image_count >= MAX_IMAGES:
        break

    full_img_path = os.path.join(IMG_DIR, rel_path)
    if not os.path.exists(full_img_path):
        print(f"Missing image: {full_img_path}")
        continue

    img = cv2.imread(full_img_path)
    if img is None:
        print(f"Failed to read image: {full_img_path}")
        continue

    faces = app.get(img)
    preds = [list(map(int, face.bbox)) for face in faces if face.det_score > 0.5]

    if len(preds) == 0 and len(gts) == 0:
        continue

    matched = set()
    for p in preds:
        hit = False
        for idx, g in enumerate(gts):
            if idx in matched:
                continue
            if iou(p, g) >= IOU_THRESH:
                TP += 1
                matched.add(idx)
                hit = True
                break
        if not hit:
            FP += 1
    FN += len(gts) - len(matched)
    image_count += 1

# STEP 6: Results
precision = TP / (TP + FP + 1e-9)
recall = TP / (TP + FN + 1e-9)
f1 = 2 * precision * recall / (precision + recall + 1e-9)

print("\n✅ SCRFD on WIDER FACE (First 100 Images) Results:")
print(f"TP: {TP}, FP: {FP}, FN: {FN}")
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")


***--CELEBA DATASET--***

In [ ]:
#Limiting CELEBA Dataset to first 3000 images

import os
import cv2
import glob
from tqdm import tqdm

# Paths
img_dir = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"
bbox_file = "/kaggle/input/celeba-dataset/list_bbox_celeba.csv"
save_dir = "/kaggle/working/celeba_haar_preds"
os.makedirs(save_dir, exist_ok=True)

# Load Haar Cascade
haar = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Parameters
scaleFactor = 1.1
minNeighbors = 5
minSize = (30, 30)

# Limit to 3000 images
img_paths = sorted(glob.glob(f"{img_dir}/*.jpg"))[:3000]

# Run detection and save in YOLO format
for img_path in tqdm(img_paths):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = haar.detectMultiScale(gray, scaleFactor=scaleFactor, minNeighbors=minNeighbors, minSize=minSize)

    h, w = img.shape[:2]
    name = os.path.splitext(os.path.basename(img_path))[0]
    with open(os.path.join(save_dir, f"{name}.txt"), "w") as f:
        for (x, y, bw, bh) in faces:
            if bw < 10 or bh < 10:
                continue
            xc = (x + bw/2) / w
            yc = (y + bh/2) / h
            bw_norm = bw / w
            bh_norm = bh / h
            f.write(f"0 {xc:.6f} {yc:.6f} {bw_norm:.6f} {bh_norm:.6f}\n")


In [ ]:
# Creating YOLO Type Labels for Annotation

import os
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt

# PART 1: Defining Paths
bbox_file = "/kaggle/input/celeba-dataset/list_bbox_celeba.csv"
img_dir = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"
label_save_dir = "/kaggle/working/celeba_yolo_labels"
os.makedirs(label_save_dir, exist_ok=True)

# PART 2: Reading bounding box CSV
df = pd.read_csv(bbox_file, skiprows=1)  # Skip the header description row
df.columns = ["image_id", "x", "y", "w", "h"]
df = df[:3000]  # Only use first 3000 images

created = 0

for index, row in df.iterrows():
    filename = row["image_id"]
    x, y, w, h = int(row["x"]), int(row["y"]), int(row["w"]), int(row["h"])

    # Skip invalid boxes
    if w <= 0 or h <= 0:
        continue

    img_path = os.path.join(img_dir, filename)
    if not os.path.exists(img_path):
        continue

    try:
        img = Image.open(img_path)
        img_w, img_h = img.size
    except:
        continue

    # Skip corrupted images
    if img_w == 0 or img_h == 0:
        continue

    # Convert to YOLO format
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w_norm = w / img_w
    h_norm = h / img_h

    # Apply epsilon *before* checking bounds
    epsilon = 1e-6
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w_norm = w / img_w
    h_norm = h / img_h

    x_center = min(max(x_center, 0), 1 - epsilon)
    y_center = min(max(y_center, 0), 1 - epsilon)
    w_norm = min(max(w_norm, 0), 1 - epsilon)
    h_norm = min(max(h_norm, 0), 1 - epsilon)

    # Now check bounds
    if not (0 <= x_center < 1 and 0 <= y_center < 1 and 0 <= w_norm < 1 and 0 <= h_norm < 1):
      continue



    label_path = os.path.join(label_save_dir, filename.replace(".jpg", ".txt"))
    with open(label_path, "w") as f:
        f.write(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
        created += 1

print(f"YOLO labels created: {created}")
print("Sample label files:", sorted(os.listdir(label_save_dir))[:5])


# PART 3: Visualization Function
def visualize_detections(img_path, label_path, detected_boxes):
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # Draw GT boxes (blue)
    if os.path.exists(label_path):
      with open(label_path, "r") as f:
        for line in f:
            print(f"Label: {line.strip()}")
            _, xc, yc, bw, bh = map(float, line.strip().split())
            x1 = int((xc - bw / 2) * w)
            y1 = int((yc - bh / 2) * h)
            x2 = int((xc + bw / 2) * w)
            y2 = int((yc + bh / 2) * h)
            print(f"GT box pixel coords: ({x1}, {y1}) to ({x2}, {y2}) on image {w}x{h}")
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)


    # Draw detections (green)
    for box in detected_boxes:
        x, y, bw, bh = box
        cv2.rectangle(img, (x, y), (x + bw, y + bh), (0, 255, 0), 2)

    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Green = Detection, Blue = Ground Truth")
    plt.show()


# PART 4: Example: Visualize for 000003.jpg
example_image = "000003.jpg"
detected_boxes = [[50, 60, 80, 100]]  # <-- placeholder detection box
visualize_detections(
    os.path.join(img_dir, example_image),
    os.path.join(label_save_dir, example_image.replace(".jpg", ".txt")),
    detected_boxes
)


In [ ]:
# Helper Functions:

import os
import numpy as np

def compute_metrics(pred_dir, label_dir, iou_thresh=0.5):
    def parse_yolo_txt(file_path):
        boxes = []
        with open(file_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    _, xc, yc, w, h = map(float, parts)
                    boxes.append([xc, yc, w, h])
        return boxes

    def yolo_to_xyxy(box, img_w, img_h):
        xc, yc, w, h = box
        x1 = int((xc - w / 2) * img_w)
        y1 = int((yc - h / 2) * img_h)
        x2 = int((xc + w / 2) * img_w)
        y2 = int((yc + h / 2) * img_h)
        return [x1, y1, x2, y2]

    def iou(boxA, boxB):
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])
        inter = max(0, xB - xA) * max(0, yB - yA)
        areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
        areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
        union = areaA + areaB - inter
        return inter / union if union > 0 else 0

    tp = fp = fn = 0
    common_files = sorted(set(os.listdir(pred_dir)) & set(os.listdir(label_dir)))
    img_size = (178, 218)  # CELEBA image size

    for fname in common_files:
        preds = parse_yolo_txt(os.path.join(pred_dir, fname))
        gts = parse_yolo_txt(os.path.join(label_dir, fname))
        preds = [yolo_to_xyxy(box, *img_size) for box in preds]
        gts = [yolo_to_xyxy(box, *img_size) for box in gts]

        matched = set()
        for p in preds:
            match_found = False
            for idx, g in enumerate(gts):
                if idx in matched:
                    continue
                if iou(p, g) >= iou_thresh:
                    tp += 1
                    matched.add(idx)
                    match_found = True
                    break
            if not match_found:
                fp += 1
        fn += len(gts) - len(matched)

    precision = tp / (tp + fp + 1e-9)
    recall = tp / (tp + fn + 1e-9)
    f1 = 2 * precision * recall / (precision + recall + 1e-9)
    return tp, fp, fn, precision, recall, f1

***HAAR CASCADES (FRONTAL + SIDE) + NMS ON CELEBA***

In [ ]:
import cv2
import os
import numpy as np
from tqdm import tqdm

# PART 1: Define Paths
img_dir = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"
save_dir = "/kaggle/working/celeba_haar_combined_nms_preds"
os.makedirs(save_dir, exist_ok=True)

# PART 2: Define Haar cascade models
frontal_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')

# PART 3: Define Parameters
CONF_THRESH = 0.5
IOU_THRESH = 0.3

# PART 4: Apply NMS
def apply_nms(boxes, scores, iou_thresh=0.3):
    if len(boxes) == 0:
        return []

    boxes_xywh = [[int(x), int(y), int(w), int(h)] for (x, y, w, h) in boxes]
    indices = cv2.dnn.NMSBoxes(boxes_xywh, scores, score_threshold=CONF_THRESH, nms_threshold=iou_thresh)

    # Flatten indices if necessary
    if isinstance(indices, np.ndarray):
        indices = indices.flatten()
    elif isinstance(indices, tuple) and len(indices) == 1 and isinstance(indices[0], (list, np.ndarray)):
        indices = indices[0]
    elif isinstance(indices, list) and isinstance(indices[0], list):
        indices = [i[0] for i in indices]

    return [boxes[i] for i in indices]


# PART 5: Run detection on first 3000 images
img_files = sorted([f for f in os.listdir(img_dir) if f.endswith(".jpg")])[:3000]

for fname in tqdm(img_files):
    img_path = os.path.join(img_dir, fname)
    img = cv2.imread(img_path)
    if img is None:
        continue

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces with proper keyword arguments
    frontal_faces = frontal_cascade.detectMultiScale(
        image=gray,
        scaleFactor=1.1,
        minNeighbors=4,
        minSize=(30, 30)
    )
    profile_faces = profile_cascade.detectMultiScale(
        image=gray,
        scaleFactor=1.1,
        minNeighbors=4,
        minSize=(30, 30)
    )

    # Combine results and apply NMS
    all_faces = list(frontal_faces) + list(profile_faces)
    scores = [0.99] * len(all_faces)
    final_faces = apply_nms(all_faces, scores, iou_thresh=IOU_THRESH)

    # Save detections in YOLO format
    h, w = img.shape[:2]
    txt_path = os.path.join(save_dir, fname.replace(".jpg", ".txt"))
    with open(txt_path, "w") as f:
        for (x, y, w_box, h_box) in final_faces:
            if w_box < 10 or h_box < 10:
                continue
            x_center = (x + w_box / 2) / w
            y_center = (y + h_box / 2) / h
            w_norm = w_box / w
            h_norm = h_box / h
            f.write(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

print("Haar cascade (frontal + profile) with NMS completed.")
print("Sample prediction files:", sorted(os.listdir(save_dir))[:5])

pred_dir = "/kaggle/working/celeba_haar_combined_nms_preds"
label_dir = "/kaggle/working/celeba_yolo_labels"

# PART 6: Compute metrics
tp, fp, fn, precision, recall, f1 = compute_metrics(pred_dir, label_dir)

# PART 7: Display results
print("Haar Cascade Evaluation on CELEBA (first 3000 images, frontal+profile+NMS)")
print(f"TP: {tp}, FP: {fp}, FN: {fn}")
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")

**YOLOV8n-FACE (without NMS) on CELEBA**

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO

# PART 1: Define Paths
img_dir = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"
label_dir = "/kaggle/working/celeba_yolo_labels"  # already created GT labels
model_path = "/root/.cache/huggingface/hub/models--arnabdhar--YOLOv8-Face-Detection/snapshots/52fa54977207fa4f021de949b515fb19dcab4488/model.pt"

# PART 2: Load Model
model = YOLO(model_path)

# PART 3: Evaluation Parameters
IOU_THRESH = 0.3
CONF_THRESH = 0.2
MIN_SIZE = 20

TP = FP = FN = 0

# PART 4: IoU Function
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    return interArea / float(boxAArea + boxBArea - interArea) if boxAArea + boxBArea - interArea > 0 else 0

# PART 5: Loop through first 3000 images
img_files = sorted(os.listdir(img_dir))[:3000]

for fname in tqdm(img_files):
    img_path = os.path.join(img_dir, fname)
    label_path = os.path.join(label_dir, fname.replace(".jpg", ".txt"))

    if not os.path.exists(label_path):
        continue  # Skip if no ground truth label

    # Read GT boxes
    gt_boxes = []
    with open(label_path, "r") as f:
        for line in f:
            parts = list(map(float, line.strip().split()[1:]))
            xc, yc, w, h = parts
            img = cv2.imread(img_path)
            h_img, w_img = img.shape[:2]
            x1 = int((xc - w / 2) * w_img)
            y1 = int((yc - h / 2) * h_img)
            w_box = int(w * w_img)
            h_box = int(h * h_img)
            gt_boxes.append([x1, y1, w_box, h_box])

    # Run YOLOv8-Face prediction
    results = model.predict(img, conf=CONF_THRESH, verbose=False)
    preds = []
    for r in results:
        for box in r.boxes:
            conf = box.conf[0].item()
            if conf < CONF_THRESH:
                continue
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            w, h = x2 - x1, y2 - y1
            if w < MIN_SIZE or h < MIN_SIZE:
                continue
            preds.append([int(x1), int(y1), int(w), int(h)])

    # Match predictions to ground truth
    matched = set()
    for p in preds:
        matched_flag = False
        for i, g in enumerate(gt_boxes):
            if i in matched:
                continue
            if iou(p, g) >= IOU_THRESH:
                TP += 1
                matched.add(i)
                matched_flag = True
                break
        if not matched_flag:
            FP += 1
    FN += len(gt_boxes) - len(matched)

# PART 6: Calculate Metrics
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print(f"YOLOv8-Face Evaluation Results on CELEBA (first 3000 images):")
print(f"TP: {TP}, FP: {FP}, FN: {FN}")
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")


**YOLOV8n-FACE (with NMS) on CELEBA**

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO

# STEP 1: Define Paths
img_dir = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"
label_dir = "/kaggle/working/celeba_yolo_labels"
model_path = "/root/.cache/huggingface/hub/models--arnabdhar--YOLOv8-Face-Detection/snapshots/52fa54977207fa4f021de949b515fb19dcab4488/model.pt"

# STEP 2: Load Model
model = YOLO(model_path)

# STEP 3: Evaluation Parameters
IOU_THRESH = 0.3
CONF_THRESH = 0.2
MIN_SIZE = 20
USE_NMS = True  # Optional NMS toggle
NMS_THRESH = 0.2
TP = FP = FN = 0

# STEP 4: IoU Function
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    return interArea / float(boxAArea + boxBArea - interArea) if boxAArea + boxBArea - interArea > 0 else 0

# STEP 5: Apply NMS
def apply_nms(boxes, scores, iou_thresh=0.4):
    if not boxes:
        return []
    boxes_xywh = [[x, y, w, h] for (x, y, w, h) in boxes]
    boxes_xyxy = [[x, y, x + w, y + h] for (x, y, w, h) in boxes]
    indices = cv2.dnn.NMSBoxes(boxes_xyxy, scores, score_threshold=0.001, nms_threshold=iou_thresh)
    if len(indices) == 0:
        return []
    indices = indices.flatten()
    return [boxes[i] for i in indices]

# STEP 6: Loop through first 3000 images
img_files = sorted(os.listdir(img_dir))[:3000]

for fname in tqdm(img_files):
    img_path = os.path.join(img_dir, fname)
    label_path = os.path.join(label_dir, fname.replace(".jpg", ".txt"))

    if not os.path.exists(label_path):
        continue

    # Load GT labels
    gt_boxes = []
    with open(label_path, "r") as f:
        for line in f:
            xc, yc, w, h = map(float, line.strip().split()[1:])
            img = cv2.imread(img_path)
            h_img, w_img = img.shape[:2]
            x1 = int((xc - w / 2) * w_img)
            y1 = int((yc - h / 2) * h_img)
            w_box = int(w * w_img)
            h_box = int(h * h_img)
            gt_boxes.append([x1, y1, w_box, h_box])

    # YOLOv8 prediction
    results = model.predict(img, conf=CONF_THRESH, verbose=False)
    boxes = []
    scores = []
    for r in results:
        for box in r.boxes:
            conf = box.conf[0].item()
            if conf < CONF_THRESH:
                continue
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            w, h = x2 - x1, y2 - y1
            if w < MIN_SIZE or h < MIN_SIZE:
                continue
            boxes.append([int(x1), int(y1), int(w), int(h)])
            scores.append(conf)

    # Optional NMS
    preds = apply_nms(boxes, scores, NMS_THRESH) if USE_NMS else boxes

    # Match predictions to ground truth
    matched = set()
    for p in preds:
        matched_flag = False
        for i, g in enumerate(gt_boxes):
            if i in matched:
                continue
            if iou(p, g) >= IOU_THRESH:
                TP += 1
                matched.add(i)
                matched_flag = True
                break
        if not matched_flag:
            FP += 1
    FN += len(gt_boxes) - len(matched)

# STEP 7: Calculate Metrics
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print(f"YOLOv8-Face Evaluation Results on CELEBA (first 3000 images, NMS={USE_NMS}):")
print(f"TP: {TP}, FP: {FP}, FN: {FN}")
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")


**SCRFD on CELEBA**

In [ ]:
import os
import cv2
import numpy as np
import onnxruntime
from tqdm import tqdm

# PART 1: Define Paths
model_path = "/content/scrfd_10g_bnkps.onnx"
img_dir = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"
label_dir = "/kaggle/working/celeba_yolo_labels"

# PART 2: Define Parameters
CONF_THRESH = 0.5
IOU_THRESH = 0.3
NMS_THRESH = 0.4
TP = FP = FN = 0
model_input_size = (640, 640)

# PART 3: Load SCRFD ONNX model
session = onnxruntime.InferenceSession(model_path, providers=['CPUExecutionProvider'])
input_name = session.get_inputs()[0].name

# PART 4: Preprocessing
def preprocess(img, size=(896, 640)):
    img_resized = cv2.resize(img, size)
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    img_norm = (img_rgb - 127.5) / 128.0
    blob = img_norm.transpose(2, 0, 1)[None].astype(np.float32)
    outputs = session.run(None, {input_name: blob})
    print([o.shape for o in outputs])
    return blob, img.shape[:2], size  # (orig_h, orig_w), input_size

# PART 5: Apply NMS
def nms(boxes, scores, threshold):
    boxes_xywh = [[x, y, w, h] for (x, y, w, h) in boxes]
    indices = cv2.dnn.NMSBoxes(boxes_xywh, scores, score_threshold=0.001, nms_threshold=threshold)
    return [boxes[i[0]] for i in indices] if isinstance(indices, (list, np.ndarray)) and len(indices) > 0 else []

# PART 6: Decode SCRFD Output
def decode_output_scrfd_multiscale(outputs, orig_h, orig_w, conf_thresh, input_size=(640, 640)):
    strides = [8, 16, 32]  # SCRFD default
    num_levels = 3
    input_w, input_h = input_size
    scale_w = orig_w / input_w
    scale_h = orig_h / input_h

    boxes = []
    confidences = []

    for i in range(num_levels):
        scores = outputs[i]
        bboxes = outputs[i + num_levels]

        for idx in range(scores.shape[0]):
            conf = float(scores[idx][0])
            if conf < conf_thresh:
                continue

            x1, y1, x2, y2 = bboxes[idx]
            x = int(x1 * scale_w)
            y = int(y1 * scale_h)
            w = int((x2 - x1) * scale_w)
            h = int((y2 - y1) * scale_h)

            boxes.append([x, y, w, h])
            confidences.append(conf)

    return boxes, confidences

# PART 7: IoU
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    return interArea / float(boxAArea + boxBArea - interArea) if boxAArea + boxBArea - interArea > 0 else 0

# PART 8: Main Evaluation Loop
img_files = sorted(os.listdir(img_dir))[:3000]

for fname in tqdm(img_files):
    img_path = os.path.join(img_dir, fname)
    label_path = os.path.join(label_dir, fname.replace(".jpg", ".txt"))

    if not os.path.exists(label_path):
        continue

    img = cv2.imread(img_path)
    blob, (orig_h, orig_w), input_size = preprocess(img, size=model_input_size)

    # ONNX Inference
    outputs = session.run(None, {input_name: blob})
    boxes, scores = decode_output_scrfd_multiscale(outputs, orig_h, orig_w, CONF_THRESH, input_size)
    print(f"{fname} → Detections: {len(boxes)}")

    import matplotlib.pyplot as plt

    for box in boxes:
        x, y, w, h = box
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f"{fname} — {len(boxes)} Detections")
    plt.axis('off')
    plt.show()
    break  # visualize only first image


    if boxes:
        boxes = nms(boxes, scores, NMS_THRESH)

    # Load GT
    gt_boxes = []
    with open(label_path, "r") as f:
        for line in f:
            xc, yc, w, h = map(float, line.strip().split()[1:])
            x1 = int((xc - w / 2) * orig_w)
            y1 = int((yc - h / 2) * orig_h)
            w_box = int(w * orig_w)
            h_box = int(h * orig_h)
            gt_boxes.append([x1, y1, w_box, h_box])

    # Match detections to GT
    matched = set()
    for pred in boxes:
        found = False
        for i, gt in enumerate(gt_boxes):
            if i in matched:
                continue
            if iou(pred, gt) >= IOU_THRESH:
                TP += 1
                matched.add(i)
                found = True
                break
        if not found:
            FP += 1
    FN += len(gt_boxes) - len(matched)

# PART 9: Calculate Metrics
precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print(f"SCRFD ONNX Evaluation on CELEBA (first 3000 images):")
print(f"TP: {TP}, FP: {FP}, FN: {FN}")
print(f"Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1:.3f}")
